#  Regressão - Preço de Venda de Diamantes

Este notebook realiza um estudo, um conjunto de experimentos, de algoritmos de regressão sobre o dataset Diamonds. Nosso objetivo é predizer o valor de um diamante baseado nas suas características.

    
Conteúdo

  
O notebook está organizado como segue:



<a href="#bibliotecas">1 - Bibliotecas</a>

<a href="#explora">2 - Análise Exploratório de dados</a>

<a href="#experimentos">3 - Experimentos</a>

<a href="#hyperparameter">4 - Hyperparameter Tuning</a>

<a href="#prevendo">5 - Prevendo o Valor de um diamante pelos atributos</a>

<a href="#conclusao">6 - Conclusão</a>


<h1 id="bibliotecas">Bibliotecas</h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV

from sklearn import preprocessing

%matplotlib inline

Função para ajustar outliers

In [ ]:

def remove_outlier(col):
    sorted(col)
    Q1,Q3=np.percentile(col,[25,75])
    IQR=Q3-Q1
    lower_range= Q1-(1.5 * IQR)
    upper_range= Q3+(1.5 * IQR)
    return lower_range, upper_range

função para normalizar dados

Variável para armazenar as métricas das experiências

In [ ]:
def scaleColumns(df, cols_to_scale):
    scaler = preprocessing.MinMaxScaler()
    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df

In [ ]:
experiment = {}

Importando os dados

In [ ]:
df_original = pd.read_csv("../input/diamonds/diamonds.csv")

In [ ]:
df_original.head()

In [ ]:
df_original = df_original.drop("Unnamed: 0", axis = 1)

<h1 id="explora">Análise Exploratória de Dados</h1>

Verificando valores nulos

In [ ]:
df_original.isna().sum()

Visualizando a distribuição dos dados 

In [ ]:
df_original.describe()

Analisando colunas categóricas

In [ ]:


df_original['color'].value_counts().plot(kind= 'bar')



In [ ]:


df_original['cut'].value_counts().plot(kind = 'bar')



In [ ]:
df_original['clarity'].value_counts()

In [ ]:
df_original.hist(figsize = (20,20), bins = 150)

In [ ]:


plt.figure(figsize=(10,10))
sns.heatmap(df_original.corr(), annot=True)



In [ ]:
sns.pairplot(df_original)

Gráfico mostra coluna preço com valores outliers

In [ ]:
plt.figure(figsize =(15,10))
df_original.boxplot(vert = 0)

In [ ]:
df = df_original.copy()
df_sem_categ = df.drop(["color","cut","clarity"], axis = 1)
scaleColumns(df_sem_categ,["carat","depth","table","x","y","z"])
df_sem_categ


Data Frame com os campos categoricos <br>
Aplicaremos a transformação de Categorico para Continuo. <br>
Não usaremos a técnica One Hot Encoding para manter os dados com um padrão ordinal sendo :<br>

**Cut**

Uma lapidação bem feita garante ao diamante um brilho que o faz se diferenciar das outras gemas. <br>

Corte       O que significa ?<br>
Excellent	Lapidação Excelente<br>
Very Good	Lapidação Muito Boa<br>
Good	    Lapidação Boa<br>
Fair	    Lapidação Aceitável<br>
Poor	    Lapidação Fraca <br>


**Carat**

O preço de um diamante diminui quanto mais cor ele apresentar.  <br>

Cor     O que significa?<br>
D	    Excepcionalmente incolor extra<br>
E	    Excepcionalmente incolor<br>
F	    Perfeitamente incolor <br>
G	    Nitidamente incolor<br>
H	    Incolor<br>
I	    Cor levemente perceptível<br>
J	    Cor perceptível<br>
K	    Cor levemente visível<br>
L	    Cor visível<br>
M - N	Cor levemente acentuada<br>
O - Z	Cor acentuada<br>

**Clarity**

O grau de pureza do diamante se refere a presença (ou não) de inclusões e manchas que possam diimuir seu valor.<br>

Grau de Pureza	O que significa ?<br>
FLAWLESS 	Internamente e externamento puro<br>
IF	        Internamente livre de inclusões<br>
VVS1 e VVS2	Inclusão ou inclusões pequeniníssimas, muito dificeis de serem visualizadas com lupa de 10x<br>
VS1 e VS2	Inclusões muito pequenas, difíceis de serem visualizadas com lupa de 10x<br>
SI1 e SI2	Inclusões pequenas, fáceis de serem visualizadas com a lupa de 10x<br>
I1	        Inclusões evidentes com lupa de 10x<br>
I2	        Uma inclusão grande ou inúmeras inclusões menores, fáceis de serem visualizadas a olho nu<br>
I3	        Uma inclusão grande ou inúmeras inclusões menores, mito fáceis de serem visualizadas a olho <br>



In [ ]:
df_com_categ_transf = df_original.copy()
df_com_categ_transf['cut'] = pd.Categorical(df['cut']).codes
df_com_categ_transf['color'] = pd.Categorical(df['color']).codes
df_com_categ_transf['clarity'] = pd.Categorical(df['clarity']).codes
scaleColumns(df_com_categ_transf,["carat","cut","color","clarity","depth","table","x","y","z"])
df_com_categ_transf

<h1 id="experimentos">Experimentos</h1>

# Experimento 01

## knn Regressor - Modelo Base
### Sem Colunas Categóricas 
### Sem Tratamento de Outliers

In [ ]:
df = df_sem_categ.copy()
x = df.drop("price", axis = 1)
y = df.pop("price")
knnmodel = KNeighborsRegressor(n_neighbors=3,metric='euclidean')
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

ini = time.time()
knnmodel.fit(x_train,y_train)
y_pred = knnmodel.predict(x_test)
fim = time.time()

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-01'] = {'Regressor':'KNN','Categoria':'Sem','Outlier':'Com', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 02
## Linear Regressor
### Sem Colunas Categóricas 
### Sem Tratamento de Outliers

In [ ]:


df = df_sem_categ.copy()
x = df.drop("price", axis = 1)
y = df.pop("price")
lrmodel = LinearRegression()
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

ini = time.time()
lrmodel.fit(x_train,y_train)
y_pred = lrmodel.predict(x_test)
fim = time.time()

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-02'] = {'Regressor':'LR','Categoria':'Sem','Outlier':'Com', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 03
## Linear Regressor  
### Sem Colunas Categóricas 
### Com Tratamento de Outliers

In [ ]:
df = df_sem_categ.copy()
for column in df.columns:
    lr,ur=remove_outlier(df[column])
    df[column]=np.where(df[column]>ur,ur,df[column])
    df[column]=np.where(df[column]<lr,lr,df[column])

Gráfico Mostra a coluna preço sem os outliers

In [ ]:
plt.figure(figsize =(15,10))
df.boxplot(vert =0)

In [ ]:
x = df.drop("price", axis = 1)
y = df.pop("price")
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

ini = time.time()
lrmodel.fit(x_train,y_train)
y_pred = lrmodel.predict(x_test)
fim = time.time()

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-03'] = {'Regressor':'LR','Categoria':'Sem','Outlier':'Sem', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 04
## Linear Regressor  
### Com Colunas Categóricas transformadas em continuas
### Sem Tratamento de Outliers

In [ ]:
df = df_com_categ_transf.copy()

x = df.drop("price", axis = 1)
y = df.pop("price")
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

ini = time.time()
lrmodel.fit(x_train,y_train)
y_pred = lrmodel.predict(x_test)
fim = time.time()


In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-04'] = {'Regressor':'LR','Categoria':'Com','Outlier':'Com', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 05
## Linear Regressor  
### Com Colunas Categóricas transformadas em continuas
### Com Tratamento de Outliers

In [ ]:
df = df_com_categ_transf.copy()
for column in df.columns:
    lr,ur=remove_outlier(df[column])
    df[column]=np.where(df[column]>ur,ur,df[column])
    df[column]=np.where(df[column]<lr,lr,df[column])

In [ ]:
x = df.drop("price", axis = 1)
y = df.pop("price")
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

ini = time.time()
lrmodel.fit(x_train,y_train)
y_pred = lrmodel.predict(x_test)
fim = time.time()

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-05'] = {'Regressor':'LR','Categoria':'Com','Outlier':'Sem', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 06
## Random Forest Regressor  
### Sem Colunas Categóricas transformadas em continuas
### Sem Tratamento de Outliers

In [ ]:
df = df_sem_categ.copy()
x = df.drop("price", axis = 1)
y = df.pop("price")
ini = time.time()
rfmodel = RandomForestRegressor(n_estimators=10, random_state=1)
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

ini = time.time()
rfmodel.fit(x_train,y_train)
y_pred = rfmodel.predict(x_test)
fim = time.time()



In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-06'] = {'Regressor':'RF','Categoria':'Sem','Outlier':'Com', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 07
## Random Forest Regressor  
### Sem Colunas Categóricas transformadas em continuas
### Com Tratamento de Outliers

In [ ]:
df = df_sem_categ.copy()
for column in df.columns:
    lr,ur=remove_outlier(df[column])
    df[column]=np.where(df[column]>ur,ur,df[column])
    df[column]=np.where(df[column]<lr,lr,df[column])

In [ ]:
x = df.drop("price", axis = 1)
y = df.pop("price")
ini = time.time()
rfmodel = RandomForestRegressor(n_estimators=10, random_state=1)
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)
rfmodel.fit(x_train,y_train)
y_pred = rfmodel.predict(x_test)
fim = time.time()

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-07'] = {'Regressor':'RF','Categoria':'Sem','Outlier':'Sem', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 08
## Random Forest Regressor  
### Com Colunas Categóricas transformadas em continuas
### Sem Tratamento de Outliers

In [ ]:
df = df_com_categ_transf.copy()
x = df.drop("price", axis = 1)
y = df.pop("price")
ini = time.time()
rfmodel = RandomForestRegressor(n_estimators=10, random_state=1)
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)
rfmodel.fit(x_train,y_train)
y_pred = rfmodel.predict(x_test)
fim = time.time()


In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-08'] = {'Regressor':'RF','Categoria':'Com','Outlier':'Com', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

# EXPERIMENTO 09
## Random Forest Regressor  
### Com Colunas Categóricas transformadas em continuas
### Com Tratamento de Outliers

In [ ]:
df = df_com_categ_transf.copy()
for column in df.columns:
    lr,ur=remove_outlier(df[column])
    df[column]=np.where(df[column]>ur,ur,df[column])
    df[column]=np.where(df[column]<lr,lr,df[column])

In [ ]:
x = df.drop("price", axis = 1)
y = df.pop("price")
ini = time.time()
rfmodel = RandomForestRegressor(n_estimators=10, random_state=1)
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)
rfmodel.fit(x_train,y_train)
y_pred = rfmodel.predict(x_test)
fim = time.time()

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-09'] = {'Regressor':'RF','Categoria':'Com','Outlier':'Sem', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

<h1 id="hyperparameter">Hyperparameter Tuning</h1>

In [ ]:
estimator = RandomForestRegressor()
para_grids = {"n_estimators" : [10,50,100,150],
              "max_features" : ["auto", "log2", "sqrt"],
              "bootstrap"    : [True, False]}

#para_grids = {"n_estimators" : [150]}


grid = GridSearchCV(estimator, para_grids)

ini = time.time()
grid.fit(x_train, y_train)
best = grid.best_estimator_
y_pred = best.predict(x_test)
fim = time.time()

In [ ]:
print ( grid.best_params_)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['EXP-10'] = {'Regressor':'RF-GS','Categoria':'Com','Outlier':'Sem', 'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'Tempo':fim-ini}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

<h1 id="prevendo">Prevendo o Valor de um diamante pelos atributos</h1>

Selecionando uma amostra da base para comparação 

In [ ]:
df = df_original.copy()
df['cut'] = pd.Categorical(df['cut']).codes
df['color'] = pd.Categorical(df['color']).codes
df['clarity'] = pd.Categorical(df['clarity']).codes
amostra = pd.DataFrame(df.loc[1250]).T
amostra

Criando um dataset para predizer

In [ ]:
anovosdados = np.array([['carat', 'cut','color','clarity','depth','table','x','y','z'], 
                       [0.72,2.0,4.0,6.0,62.5,57.0,5.7,5.73,3.57]])
                   
dfnovosdados = pd.DataFrame(anovosdados)
dfnovosdados.columns = anovosdados[0]
dfnovosdados = dfnovosdados.drop(0)
dfnovosdados

Padronizando os dados 

In [ ]:
x = df.drop("price", axis = 1)
scaler = preprocessing.MinMaxScaler()
scaler.fit(x)
x1 = scaler.transform(dfnovosdados)
print(x1)

In [ ]:
predicao = best.predict(x1)

Comparando valores (Real x Previsto pelos atributos)

In [ ]:
valor_real = amostra.iloc[0]["price"]
valor_previsto = predicao[0]
print('Valor do Diamante Real: ',valor_real)
print('Valor do Diamante Previsto: ',valor_previsto)

In [ ]:
exp = pd.DataFrame(experiment).T

exp.sort_index()

In [ ]:
plt.figure(figsize=(10,2))
sns.barplot(x='R2' , y='Regressor' , data=exp)

<h1 id="conclusao">Conclusão</h1>

É necessário um conhecimento do negócio para entendermos como devemos tratar as colunas. <br>
Os algoritmos treinados com a inclusão das colunas categoricas transformadas em valores continuos apresentaram melhor performance.<br>

Algoritmo RandomForestRegressor apresentou melhor peformance comparado ao LinearRegression e KNN
mas a análise do tempo de execução mostrou um custo computacional muito superior.<br>

A exclusão dos valores outliers melhoraram o processo de treinamento o que nos mostra a importãncia da análise descritiva para a identificação de problemas e a correção dos dados apresentados ao algoritmos e que somente a escolha do algoritmo, mesmo o melhor algoritmo, não é capaz de apresentar os melhores resultados possíveis. 